# Setup


In [1]:
import os
from openai import OpenAI

import utils
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# OpenAI Function Calling


In [2]:
import json


# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [3]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [4]:
messages = [{"role": "user", "content": "What's the weather like in Boston?"}]

In [5]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613", messages=messages, functions=functions
)

In [9]:
utils.print_as_json(response)

{
    "id": "chatcmpl-9UftfNVZpVyYIaSNfBH1cPj4bf2Pj",
    "choices": [
        {
            "finish_reason": "function_call",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": null,
                "role": "assistant",
                "function_call": {
                    "arguments": "{\n  \"location\": \"Boston, MA\"\n}",
                    "name": "get_current_weather"
                },
                "tool_calls": null
            }
        }
    ],
    "created": 1717097919,
    "model": "gpt-3.5-turbo-0613",
    "object": "chat.completion",
    "system_fingerprint": null,
    "usage": {
        "completion_tokens": 18,
        "prompt_tokens": 82,
        "total_tokens": 100
    }
}


In [10]:
response_message = response.choices[0].message

In [15]:
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None)

In [14]:
response_message.content

In [13]:
response_message.function_call

FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather')

In [ ]:
json.loads(response_message.function_call.arguments)

{'location': 'Boston, MA'}

In [16]:
args = json.loads(response_message.function_call.arguments)

In [17]:
get_current_weather(args)

'{"location": {"location": "Boston, MA"}, "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

### Test non related messages


In [18]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

In [19]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

In [21]:
utils.print_as_json(response)

{
    "id": "chatcmpl-9Ug2cbQSZOGyPQmr3SpzTZNnjn7hn",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "Hello! How can I assist you today?",
                "role": "assistant",
                "function_call": null,
                "tool_calls": null
            }
        }
    ],
    "created": 1717098474,
    "model": "gpt-3.5-turbo-0613",
    "object": "chat.completion",
    "system_fingerprint": null,
    "usage": {
        "completion_tokens": 10,
        "prompt_tokens": 76,
        "total_tokens": 86
    }
}


In [22]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
utils.print_as_json(response)

{
    "id": "chatcmpl-9Ug32SrlflaI6BwwwKvdJAKbqpxDv",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "Hello! How can I assist you today?",
                "role": "assistant",
                "function_call": null,
                "tool_calls": null
            }
        }
    ],
    "created": 1717098500,
    "model": "gpt-3.5-turbo-0613",
    "object": "chat.completion",
    "system_fingerprint": null,
    "usage": {
        "completion_tokens": 10,
        "prompt_tokens": 76,
        "total_tokens": 86
    }
}


In [23]:
# Test with function_call="none"
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
utils.print_as_json(response)

{
    "id": "chatcmpl-9Ug4LBrA1Z5iKVD2DiT53FhpAjLeT",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "Hello! How can I assist you today?",
                "role": "assistant",
                "function_call": null,
                "tool_calls": null
            }
        }
    ],
    "created": 1717098581,
    "model": "gpt-3.5-turbo-0613",
    "object": "chat.completion",
    "system_fingerprint": null,
    "usage": {
        "completion_tokens": 9,
        "prompt_tokens": 77,
        "total_tokens": 86
    }
}


In [25]:
# Test with function_call="none"
messages = [
    {
        "role": "user",
        "content": "What's the weather in Boston?",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
utils.print_as_json(response)

{
    "id": "chatcmpl-9Ug5IQOg5AYWb7DQOwq955B6VTyJU",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "The weather in Boston is currently cloudy with a temperature of 12 degrees Celsius.",
                "role": "assistant",
                "function_call": null,
                "tool_calls": null
            }
        }
    ],
    "created": 1717098640,
    "model": "gpt-3.5-turbo-0613",
    "object": "chat.completion",
    "system_fingerprint": null,
    "usage": {
        "completion_tokens": 16,
        "prompt_tokens": 82,
        "total_tokens": 98
    }
}


In [26]:
# Test forcing to call the function (    function_call={"name": "get_current_weather"},)
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
utils.print_as_json(response)

{
    "id": "chatcmpl-9Ug5cYXnqSsGpSN01fpaXsKG1kxt9",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": null,
                "role": "assistant",
                "function_call": {
                    "arguments": "{\n  \"location\": \"San Francisco, CA\"\n}",
                    "name": "get_current_weather"
                },
                "tool_calls": null
            }
        }
    ],
    "created": 1717098660,
    "model": "gpt-3.5-turbo-0613",
    "object": "chat.completion",
    "system_fingerprint": null,
    "usage": {
        "completion_tokens": 12,
        "prompt_tokens": 83,
        "total_tokens": 95
    }
}


In [27]:
# Test forcing to call the function (    function_call={"name": "get_current_weather"},)

messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
utils.print_as_json(response)

{
    "id": "chatcmpl-9Ug65GUGgOf0wCNGA3BUWWMdDSfrM",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": null,
                "role": "assistant",
                "function_call": {
                    "arguments": "{\n  \"location\": \"Boston, MA\"\n}",
                    "name": "get_current_weather"
                },
                "tool_calls": null
            }
        }
    ],
    "created": 1717098689,
    "model": "gpt-3.5-turbo-0613",
    "object": "chat.completion",
    "system_fingerprint": null,
    "usage": {
        "completion_tokens": 11,
        "prompt_tokens": 89,
        "total_tokens": 100
    }
}


In [28]:
messages.append(response.choices[0].message)

In [29]:
args = json.loads(response.choices[0].message.function_call.arguments)
observation = get_current_weather(args)

In [30]:
messages.append(
    {
        "role": "function",
        "name": "get_current_weather",
        "content": observation,
    }
)

In [31]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
)
utils.print_as_json(response)

{
    "id": "chatcmpl-9Ug6wvpk1fdKqypkXDDTcyuMUoRX9",
    "choices": [
        {
            "finish_reason": "stop",
            "index": 0,
            "logprobs": null,
            "message": {
                "content": "The current weather in Boston, MA is 72\u00b0F (22\u00b0C). The forecast for today is sunny and windy.",
                "role": "assistant",
                "function_call": null,
                "tool_calls": null
            }
        }
    ],
    "created": 1717098742,
    "model": "gpt-3.5-turbo-0613",
    "object": "chat.completion",
    "system_fingerprint": null,
    "usage": {
        "completion_tokens": 24,
        "prompt_tokens": 77,
        "total_tokens": 101
    }
}
